In [1]:
# % load_ext autoreload
# % autoreload 2

In [2]:
import lightkurve as lk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from patsy import dmatrix
from lightkurve.correctors import DesignMatrix, DesignMatrixCollection

In [3]:
lcf = lk.search_lightcurvefile('K2-50').download()

In [4]:
lc = lcf.PDCSAP_FLUX.remove_nans().normalize()

In [25]:
sff = lc.remove_nans().to_corrector()

In [26]:
# %load_ext line_profiler

In [29]:
clc = sff.correct()

In [24]:
r = lk.correctors.RegressionCorrector(lc)

In [12]:
%lprun -f sff.X[0]._validate sff.X[0]._validate()

UsageError: Line magic function `%lprun` not found.


In [ ]:
np.linalg.matrix_rank(sff.X[0].values)

In [ ]:
sff.X.shape

In [ ]:
plt.pcolormesh(sff.X[0].values.T)

In [ ]:
%lprun -f r.correct r.correct(sff.X)

In [ ]:
clc.errorbar()

In [ ]:
ax = lc.flatten(101).errorbar()
clc.normalize().errorbar(ax=ax)

In [ ]:
plt.scatter(lcf.PDCSAP_FLUX.remove_nans().normalize().flux_err, clc.flux_err)

In [ ]:
corrected_lc = sff.correct(centroid_col=centroid_col,
                           centroid_row=centroid_row, windows=2, bins=5, restore_trend=True, cadence_mask=mask, breakindex=150)

In [ ]:
ax = lc.plot(label='Raw')
corrected_lc.plot(ax=ax, lw=2, label='lk.SFFCorrector')
plt.plot(time, corrected_flux - np.median(corrected_flux) + 1, label='Vanderburg')
plt.legend()

In [ ]:
n_points = 300
raw_flux = data[:, 1][:n_points]
centroid_col = data[:, 3][:n_points]
centroid_row = data[:, 4][:n_points]

time = np.concatenate((np.linspace(0, 20, int(n_points/3)),
                       np.linspace(30, 78, int(n_points/3)),
                       np.linspace(80, 100, int(n_points/3))
                       ))
lc = lk.KeplerLightCurve(time=time,
                      flux=raw_flux,
                      flux_err=np.ones(n_points) * 0.0001,
                      centroid_col=centroid_col,
                      centroid_row=centroid_row)


In [ ]:
lc.errorbar()

In [ ]:
lk.correctors.SFFCorrector(lc).correct()

In [ ]:
np.isclose(corrected_flux, corrected_lc.flux, atol=0.5e-3).all()

In [ ]:
#tpf = lk.search_targetpixelfile('K2-43').download()
lcf = lk.search_lightcurvefile('K2-43').download()

In [ ]:
def get_spline_dm(x, n_knots=20, degree=3, name='spline', include_intercept=False):
    spline_dm = np.asarray(dmatrix("bs(x, df={}, degree={}, include_intercept={}) - 1".format(n_knots, degree, include_intercept), {"x": x}))
    df = pd.DataFrame(spline_dm, columns=['knot{}'.format(idx + 1) for idx in range(n_knots)])
    return DesignMatrix(df, name=name)

def get_centroid_dm(c, r, name='centroids'):
    arclength = ((c - c.min())**2 + (r - r.min())**2)**0.5

    ar = [c, r,
          c**2, r**2,
          c**3, r**3,
          c*r,
          c**2*r, c*r**2, c**2*r**2]

    columns = [r'col', r'row',
               r'col^2', r'row^2',
               r'col^3', r'row^3',
               r'col \times row', r'col^2 \times row', r'col \times row^2', r'col^2 \times row^2']

    df = pd.DataFrame(np.asarray(ar).T, columns=columns)
    return DesignMatrix(df, name=name)

def get_pixel_dm(pixels):
    pass

In [ ]:
from astropy.modeling import models, fitting
def _get_window_points(self, windows, arc, breakindex):
        ''' Build window points, based on where thrusters are fired. '''

        def _get_thruster_firings(arclength):
            ''' Find locations where K2 fired thrusters
            Parameters:
            ----------
            arc : np.ndarray
                arclength as a function of time
            Returns:
            -------
            thrusters: np.ndarray of bools
                True at times where thrusters were fired.
            '''
            arc = np.copy(arclength)
            # Rate of change of rate of change of arclength wrt time
            d2adt2 = (np.gradient(np.gradient(arc)))
            # Fit a nice Gaussian, most points lie in a tight region, thruster firings are outliers
            g = models.Gaussian1D(amplitude=100, mean=0, stddev=0.01)
            fitter = fitting.LevMarLSQFitter()
            h = np.histogram(d2adt2[np.isfinite(d2adt2)], np.arange(-0.5, 0.5, 0.0001), density=True);
            xbins = h[1][1:] - np.median(np.diff(h[1]))
            g = fitter(g, xbins, h[0], weights=h[0]**0.5)

            def _start_and_end(method):
                ''' Find points at the start or end of a roll
                '''
                if method == 'start':
                    thrusters = (d2adt2 < (g.stddev * -5)) & np.isfinite(d2adt2)
                if method == 'end':
                    thrusters = (d2adt2 > (g.stddev * 5)) & np.isfinite(d2adt2)
                # Pick the best thruster in each cluster
                idx = np.array_split(np.arange(len(thrusters)), np.where(np.gradient(np.asarray(thrusters, int)) == 0)[0])
                m = np.array_split(thrusters, np.where(np.gradient(np.asarray(thrusters, int)) == 0)[0])
                th = []
                for jdx in range(len(idx)):
                    if m[jdx].sum() == 0:
                        th.append(m[jdx])
                    else:
                        th.append((np.abs(np.gradient(arc)[idx[jdx]]) == np.abs(np.gradient(arc)[idx[jdx]][m[jdx]]).max()) & m[jdx])
                thrusters = np.hstack(th)
                return thrusters

            # Get the start and end points
            thrusters = np.asarray([_start_and_end('start'), _start_and_end('end')])
            thrusters = thrusters.any(axis=0)

            # Take just the first point.
            thrusters = (np.gradient(np.asarray(thrusters, int)) >= 0) & thrusters
            return thrusters

        thrusters = _get_thruster_firings(arc)
        thrusters[breakindex] = True
        thrusters = np.where(thrusters)[0]

        if breakindex != 0:
            window_points = np.append(np.linspace(0, breakindex + 1, windows//2 + 1, dtype=int)[1:],
                                  np.linspace(breakindex + 1, len(arc), windows//2 + 1, dtype=int)[1:-1])
            window_points[np.argmin((window_points - breakindex + 1)**2)] = breakindex + 1
        else:
            window_points = np.linspace(0, len(self.flux) + 1, windows)
        window_points = [thrusters[np.argmin(np.abs(wp - thrusters))] + 1 for wp in window_points]

        if window_points[0] < (np.median(np.diff(window_points)) * 0.4):
            window_points = window_points[1:]
        return window_points


In [ ]:
lc = lcf.PDCSAP_FLUX.remove_nans()

In [ ]:
c, r = lc.centroid_col - lc.centroid_col.mean(), lc.centroid_row  - lc.centroid_row.mean()
arclength = ((c - c.min())**2 + (r - r.min())**2)**0.5
dm = lk.correctors.DesignMatrixCollection([get_centroid_dm(c, r).append_constant().split(1753), get_spline_dm(lc.time, n_knots=80)])

In [ ]:
dm.plot()

In [ ]:
r = lk.correctors.RegressionCorrector(lc, dm)
r.correct(niters=5)
r.diagnose()

In [ ]:
# def get_sff_dm(lc, breakindex=1753, windows=5, bins=5, n_knots=80):
#     c, r = lc.centroid_col - lc.centroid_col.mean(), lc.centroid_row  - lc.centroid_row.mean()
#     arclength = ((c - c.min())**2 + (r - r.min())**2)**0.5

#     # centroids
#     c_dm = get_centroid_dm(c, r).split(1753).whiten()

#     # long term
#     lt_dm = get_spline_dm(lc.time, n_knots=n_knots, include_intercept=True)

#     # sff
#     window_points = _get_window_points(lc, windows, arclength, breakindex)
#     sff_dm = get_spline_dm(arclength, n_knots=bins, name='arclength', degree=3, include_intercept=True)
    
    
#     return DesignMatrixCollection([lt_dm, sff_dm.split(window_points)]), window_points

In [ ]:
windows = 20
bins = 5
degree = 3
include_intercept=False
breakindex=1753
wp = _get_window_points(lc, windows, arclength, breakindex)


lower_idx = np.append(0, wp)
upper_idx = np.append(wp, len(lc.time))

stack = []
columns = []
for idx, a, b in zip(range(len(lower_idx)), lower_idx, upper_idx):
    #knots = list(np.linspace(*np.percentile(arclength[a:b], [5, 95]), bins))
    knots = list(np.percentile(arclength[a:b], np.linspace(0, 100, bins+1)[1:-1]))
    ar = np.copy(arclength)
    ar[~np.in1d(ar, ar[a:b])] = -1
    dm = np.asarray(dmatrix("bs(x, knots={}, degree={}, include_intercept={}) - 1".format(knots, degree, include_intercept), {"x": ar}))
    stack.append(dm)
    columns.append(['window{}_bin{}'.format(idx+1, jdx+1) for jdx in range(len(dm.T))])


sff_dm = DesignMatrix(pd.DataFrame(np.hstack(stack), columns=np.hstack(columns)), name='arclength')

# long term
s_dm = get_spline_dm(lc.time, n_knots=100, include_intercept=True)


#c_dm = DesignMatrix(pd.DataFrame(np.asarray([lc.centroid_col, lc.centroid_row]).T, columns=['col', 'row']), name='c')
c_dm = get_centroid_dm(lc.centroid_col, lc.centroid_row)
dm = DesignMatrixCollection([c_dm, s_dm.append_constant(), sff_dm])

In [ ]:
r = lk.correctors.RegressionCorrector(lc, dm)
r.correct(niters=5)

In [ ]:
r.corrected_lc.estimate_cdpp()

In [ ]:
r.diagnose()

In [ ]:
r.diagnostic_lightcurves['spline'].plot()

In [ ]:
# lower_idx = np.append(0, wp)
# upper_idx = np.append(wp, len(lc.time))
# f = (lc.flux - r.diagnostic_lightcurves['spline'].flux)
# m = r.diagnostic_lightcurves['arclength'].flux

# for a, b in zip(lower_idx, upper_idx):
#     #plt.figure()
#     plt.scatter(arclength[a:b], f[a:b], s=1)
#     plt.scatter(arclength[a:b][~r.cadence_mask[a:b]], f[a:b][~r.cadence_mask[a:b]], s=10, marker='x')

#     s = np.argsort(arclength[a:b])
#     plt.plot(arclength[a:b][s], (m[a:b] - np.median(m[a:b]) + np.median(f[a:b]))[s])
#     plt.xlabel('Arclength')
#     plt.ylabel('Flux')
#     #break

In [ ]:
# lower_idx = np.append(0, wp)
# upper_idx = np.append(wp, len(lc.time))
# f = (lc.flux - r.diagnostic_lightcurves['spline'].flux)
# m = r.diagnostic_lightcurves['arclength'].flux

# for a, b in zip(lower_idx, upper_idx):
#     plt.figure()
#     plt.scatter(arclength[a:b], f[a:b], c='k', s=1)
#     plt.scatter(arclength[a:b][~r.cadence_mask[a:b]], f[a:b][~r.cadence_mask[a:b]], c='r', s=10, marker='x')

#     s = np.argsort(arclength[a:b])
#     plt.plot(arclength[a:b][s], (m[a:b] - np.median(m[a:b]) + np.median(f[a:b]))[s], c='b')
#     plt.xlabel('Arclength')
#     plt.ylabel('Flux')
#     #break

In [ ]:
plt.scatter(arclength, lc.flux - r.diagnostic_lightcurves['spline'].flux, s=1, c='k')
plt.scatter(arclength, lc.flux - r.diagnostic_lightcurves['spline'].flux, s=1, c='k')

In [ ]:
r.X.plot()

In [ ]:
np.hstack([d.columns for d in dm])